In [1]:
import requests
import os
from bs4 import BeautifulSoup
import re

import urllib.parse

import time
import pickle

In [2]:
siteUrl = "http://www.commercialtrucktrader.com/Heavy-Duty-Trucks-For-Sale/search-results"
siteUrl = "?".join([siteUrl, "type=heavy"])
print(siteUrl)

proxies = {'http': 'http://<<USERID>>:<<PASSWORD>>@23.19.51.245:40791/'}
max_page = 0
seen_urls_array = []

http://www.commercialtrucktrader.com/Heavy-Duty-Trucks-For-Sale/search-results?type=heavy


In [3]:
#<a href="/Heavy-Duty-Trucks-For-Sale/search-results?type=heavy&amp;page=1799" class="listings-pag-default">1799</a>

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=siteUrl, proxies=proxies)
sitePageSoup = BeautifulSoup(sitePage.content, 'lxml')

css = 'div.listings-pag-bottom > div > a.listings-pag-default'
pageLinks = sitePageSoup.select(css)
if len(pageLinks):
    max_page = int(pageLinks[-1]["href"].split('=')[2])
print(max_page)

1719


In [4]:
for i in range(1, max_page+1):
    tagUrl = "&".join([siteUrl, "page="+str(i)])
    page = requests.get(url=tagUrl, proxies=proxies)
    soup = BeautifulSoup(page.content, 'lxml')

    # Get the title of the page to prove we are progressing
    titleTag = soup.select("head > title")
    titleString = titleTag[0].get_text().strip()
    #print(titleString)

    # Get the links to the individual pages
    pageLinks = soup.select('div.listing-header > a > h2')
    for pageLink in pageLinks:
        pageUrl = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', pageLink.parent['href']))
        #print(pageUrl)
        if pageUrl not in seen_urls_array:
            seen_urls_array.append(pageUrl)
    
    # Destroy the tree when you're done working with it
    soup.decompose()

# Show the number of individual page links you have
print(len(seen_urls_array))

42935


In [5]:
with open('commercialtrucktraderHeavyDutyTrucks.pickle', 'wb') as handle:
    pickle.dump(seen_urls_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
import pickle

with open('commercialtrucktraderHeavyDutyTrucks.pickle', 'rb') as handle:
    seen_urls_array = pickle.load(handle)

In [7]:
from random import randint

pageUrl = seen_urls_array[randint(0,len(seen_urls_array)-1)]
print(pageUrl)

http://www.commercialtrucktrader.com/listing/2011-Freightliner-Columbia-120--120787998


In [8]:
from selenium import webdriver
from base64 import b64encode

proxy = {'host': '23.19.51.245', 'port': '40791', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>'}
# Retrieve the page with tag results and set it up to be scraped
fp = webdriver.FirefoxProfile()
fp.set_preference('network.proxy.type', 1)
fp.set_preference('network.proxy.http', proxy['host'])
fp.set_preference('network.proxy.http_port', int(proxy['port']))
fp.set_preference('network.proxy.no_proxies_on', 'localhost, 127.0.0.1')
fp.add_extension('closeproxy.xpi')
credentials = '{usr}:{pwd}'.format(**proxy)
credentials = b64encode(credentials.encode('ascii')).decode('utf-8')
fp.set_preference('extensions.closeproxyauth.authtoken', credentials)

#set some privacy settings
fp.set_preference( "places.history.enabled", False )
fp.set_preference( "privacy.clearOnShutdown.offlineApps", True )
fp.set_preference( "privacy.clearOnShutdown.passwords", True )
fp.set_preference( "privacy.clearOnShutdown.siteSettings", True )
fp.set_preference( "privacy.sanitize.sanitizeOnShutdown", True )
fp.set_preference( "signon.rememberSignons", False )
fp.set_preference( "network.cookie.lifetimePolicy", 2 )
fp.set_preference( "network.dns.disablePrefetch", True )
fp.set_preference( "network.http.sendRefererHeader", 0 )

#if you're really hardcore about your security
#js can be used to reveal your true i.p.
fp.set_preference( "javascript.enabled", False )

#get a huge speed increase by not downloading images
fp.set_preference( "permissions.default.image", 2 )

driver = webdriver.Firefox(fp)

In [9]:
import time

# set timeout information
driver.set_page_load_timeout(5)

finished = 0
while finished == 0:
    try:
        driver.get(pageUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        time.sleep(5)

In [11]:
soup = BeautifulSoup(driver.page_source, 'lxml')

seller = soup.select('div.details-right > strong')
if len(seller):
    seller = seller[0].get_text().strip()
else:
    seller = "Unknown"
print(seller)

model = soup.select("div.details > h1")
if len(model):
    model = model[0].get_text().strip()
else:
    model = "Unknown"
print(model)

price = soup.select("div.details-right > h2.lfloat")
if len(price):
    price = price[0].get_text().strip()
else:
    price = "Unknown"
print(price)

phone = soup.select("span.flipphone.bold.font1-1")
if len(phone):
    phone = phone[0].get_text().strip()[::-1]
else:
    phone = ""
print(phone)

G  and  J Truck Sales
2011 FREIGHTLINER COLUMBIA 120 CONVENTIONAL - SLEEPER TRUCK in Groom, TX
Price: $16,900
)866( 565-0320


In [ ]:
for pageUrl in seen_urls_array:
    try:
        finished = 0
        while finished == 0:
            try:
                driver.get(pageUrl)
                finished = 1
            except Exception as e:
                print(str(e))
                time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

        seller = soup.select('div.details-right > strong')
        if len(seller):
            seller = seller[0].get_text().strip()
        else:
            seller = "Unknown"
        #print(seller)

        model = soup.select("div.details > h1")
        if len(model):
            model = model[0].get_text().strip()
        else:
            model = "Unknown"
        #print(model)

        price = soup.select("div.details-right > h2.lfloat")
        if len(price):
            price = price[0].get_text().strip()
        else:
            price = "Unknown"
        #print(price)

        phone = soup.select("span.flipphone.bold.font1-1")
        if len(phone):
            phone = phone[0].get_text().strip()[::-1]
        else:
            phone = ""
        #print(phone)
        
        if len(phone):
            bigFile = open('commercialtrucktraderHeavyDutyTrucks.txt', 'a', encoding='utf-8')
            bigFile.write(seller + ': ' + model + '\t' + price + '\t' + phone + '\n')
            bigFile.close()
    
        # Destroy the tree when you're done working with it
        soup.decompose()
        
    except Exception as e:
        print(str(e))

In [12]:
driver.quit()